In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,038 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:1

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-30 00:00:14--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.27MB/s    in 0.7s    

2022-11-30 00:00:16 (1.27 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
video_games_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
# Read in data from S3 Buckets
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
appliances_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
appliances_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...| 2015-08-31|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|Major Appliances|          5|    

In [6]:
# Create review ID Table with columns Review_ID, Customer_ID, Product_ID, Product_Parent, and Review_Date
# change date format <---------------------------------
from pyspark.sql.functions import col
from pyspark.sql.functions import to_date
Review_ID_df = video_games_df.select(["review_id","customer_id","product_id","product_parent","review_date"])
Review_ID_df = Review_ID_df.withColumn("review_date",to_date(col('review_date'), 'yyyy-mm-dd'))
Review_ID_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-01-31|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175| 2015-01-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-01-31|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350| 2015-01-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-01-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-01-31|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603| 2015-01-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-01-31|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890| 2015-01-31|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651| 2015-01-31|
|R1VTIA3JTYBY02|     106569|B00008KTNN|     384411423| 2015-01-31|
|R29DOU8791QZL8|   48269642|B000A3IA0Y|     472622859| 2015-01

In [7]:
Review_ID_df = Review_ID_df.drop_duplicates()
Review_ID_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1KLINKRR2K3FG|   32344804|B00AXE639A|     705151829| 2015-01-31|
| RVW31DRKPTBIO|   40265615|B00BMFIXZG|     175986677| 2015-01-31|
|R3NNSB8LL3I53Z|   22429897|B004UPP5TC|     498076313| 2015-01-31|
|R3K4V7YIICUNWJ|   17649709|B00KWEH61U|     171464234| 2015-01-30|
|R2Y3BUO9LTEP5K|   11076595|B00YDFM3FI|     720817043| 2015-01-30|
| RJOGGZXL9V6P7|    3439305|B00OZBQIL0|     353072225| 2015-01-30|
|R32BS7M6G3GU1D|   44476363|B00Z0DOA9O|      55836087| 2015-01-30|
|R2MF9TO2LNDF15|   14968294|B00RSXRLUE|     490486766| 2015-01-30|
|R355WJYUOBKN6O|   26953574|B00OBZNI0O|     773772231| 2015-01-30|
|R19EX7P7H3268P|   45666823|B00FLLFJ68|     942632924| 2015-01-30|
|R1G94LIV8H0GG8|   37937763|B00HM3QANO|     340945032| 2015-01-29|
| RA7EEAA7MSSKZ|   15102619|B00BMFIXKQ|     182750956| 2015-01

In [24]:
Review_ID_df.count()

1785997

In [8]:
clean_rev_id_df = Review_ID_df.groupBy("review_id").count().filter("count > 1")
clean_rev_id_df.show()

+---------+-----+
|review_id|count|
+---------+-----+
+---------+-----+



In [9]:
# Create a Product Table with columns Product_ID, and Product_Title
Product_ID_df = video_games_df.select(["product_id","product_title"])
Product_ID_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001CXYMFS|Thrustmaster T-Fl...|
|B00M920ND6|Tonsee 6 buttons ...|
|B0029CSOD2|Hidden Mysteries:...|
|B00GOOSV98|GelTabz Performan...|
|B00Y074JOM|Zero Suit Samus a...|
|B002UBI6W6|Psyclone Recharge...|
|B008XHCLFO|Protection for yo...|
|B00BRA9R6A|   Nerf 3DS XL Armor|
|B009EPWJLA|One Piece: Pirate...|
|B0000AV7GB|Playstation 2 Dan...|
|B00008KTNN|Metal Arms: Glitc...|
|B000A3IA0Y|72 Pin Connector ...|
|B0053BQN34|uDraw Gametablet ...|
|B002I0HIMI|NBA 2K12(Covers M...|
|B0081EH12M|New Trigger Grips...|
|B005FMLZQQ|Xbox 360 Media Re...|
|B002BSA388|Super Mario Galaxy 2|
|B00BUSLSAC|Nintendo 3DS XL -...|
|B00KWF38AW|Captain Toad:  Tr...|
|B00BRQJYA8|Lego Batman 2: DC...|
+----------+--------------------+
only showing top 20 rows



In [25]:
Product_ID_df.count()

1785997

In [10]:
clean_product_id_df = Product_ID_df.groupBy("product_id").count().filter("count > 1")
clean_product_id_df.show()

+----------+-----+
|product_id|count|
+----------+-----+
|B00TS0UKE4|  119|
|B004LQRF3Y|  142|
|B002I0K3Z2|  518|
|B002ZKTHD4|   83|
|B00BMFIXT2| 1467|
|B00XMQBA62|   18|
|B00N4ABMG4|  136|
|B00IEYG1AC|   29|
|B006VB2UNM| 1051|
|B00HPU96FI|   76|
|B00HSY1R3K|   14|
|B002BZ11E6|   39|
|B000YQ32TG|   39|
|B002EE1P2W|  159|
|B006WQR3GA|  311|
|B00005MDZ8|  155|
|B00002STA7|    6|
|B000P5BSUQ|  190|
|B0000296ZL|   51|
|B00DBCAS7E|   75|
+----------+-----+
only showing top 20 rows



In [14]:
test_product_id_df = Product_ID_df.dropDuplicates()

In [15]:
test_product_id_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CJ7IUI6|The Elder Scrolls...|
|B00DHF39KS|Wolfenstein: The ...|
|B00MUTAVH6|Under Night In-Bi...|
|B001AZSEUW|              Peggle|
|B00KVOVBGM|PlayStation 4 Con...|
|B00O9VGH4Y|USPRO&reg; Headph...|
|B004OQNZY4|Phineas and Ferb:...|
|B00ZLN980O|Donop seablue 2.4...|
|B002L8W5V6|Dotop Nintendo Ga...|
|B007AJZ5PY|Nyko Game Case fo...|
|B000AOEU2K|Fire Emblem: Path...|
|B000H8BW7U|Tanarus (PC) (Com...|
|B013RADQOQ|Susenstone® 2400D...|
|B00KQXKUJ2|FIFA 15 (Ultimate...|
|B006W41X2C|Turtle Beach - Ea...|
|B000KCX9M4|Grand Theft Auto:...|
|B00YT90JWC|Red Wii Mini Cons...|
|B0096KG6A8|Wii U Super Mario...|
|B00L6AVLB0|World of Tanks-X3...|
|B000IMYKQ0|Wii Nunchuk Contr...|
+----------+--------------------+
only showing top 20 rows



In [16]:
no_dupe_product_id_df = test_product_id_df.groupby("product_id").count().filter("count > 1")

In [17]:
no_dupe_product_id_df.show()

+----------+-----+
|product_id|count|
+----------+-----+
+----------+-----+



In [18]:
# Create a Customers Table with columns Customer_ID, and Customer_Count
Customers_df = video_games_df.groupby("customer_id").agg({"customer_id":"count"})
Customers_df.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   48670265|                 1|
|   49103216|                 2|
|    1131200|                 1|
|   43076447|                 2|
|   46261368|                 1|
|    4883305|                 5|
|   41192649|                 1|
|   40985731|                 7|
|   10437900|                 2|
|   22245671|                 1|
|    2574873|                 1|
|    4696154|                 1|
|    5621202|                 1|
|    5871933|                 2|
|   44089812|                 1|
|    2845910|                 1|
|    5274369|                 1|
|   39069693|                 2|
|     137793|                 1|
|   31914942|                 3|
+-----------+------------------+
only showing top 20 rows



In [26]:
Customers_df.count()

1045733

In [19]:
Customers_df = Customers_df.withColumnRenamed("count(customer_id)", "customer_count")

In [20]:
# Vine Table with columns Review_ID, Star_Rating, Helpful_Votes, Total_Votes, and Vine
Vine_df = video_games_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
Vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RTIS3L2M1F5SM|          5|            0|          0|   N|
| R1ZV7R40OLHKD|          5|            0|          0|   N|
|R3BH071QLH8QMC|          1|            0|          1|   N|
|R127K9NTSXA2YH|          3|            0|          0|   N|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|
|R2F0POU5K6F73F|          5|            0|          0|   N|
|R3VNR804HYSMR6|          5|            0|          0|   N|
| R3GZTM72WA2QH|          5|            0|          0|   N|
| RNQOY62705W1K|          4|            0|          0|   N|
|R1VTIA3JTYBY02|          5|            0|          0|   N|
|R29DOU8791QZL8|          1|            0|          0|   N|
|R15DUT1VIJ9RJZ|          2|            0|          0|   N|
|R3IMF2MQ3OU9ZM|          4|            

In [27]:
Vine_df.count()

1785997

In [21]:
# Load into Postgres
#Configure Postgres and Colab together
mode = "append"
jdbc_url="jdbc:postgresql://amazon-reviews.cfiruawf1lll.us-east-1.rds.amazonaws.com:5432/video_games_db"
config = {"user":"postgres", "password": "", "driver":"org.postgresql.Driver"}

In [22]:
#redo four times one time for each table
Review_ID_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
Review_ID_df.count()

In [23]:
#redo four times one time for each table
test_product_id_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
Customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
Vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

It looks like there are less customers than reviews. So that suggests that customers who do write reviews will have a greater probablity to write another.